In [ ]:
####################################################
# Scraping Data From Twitter HTML Files
#
# 1. Place HTML files in 'HTML' directory in CWD.
# 2. 
#
####################################################

In [ ]:
import os
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import pymongo

In [ ]:
mongo_connect_string = 'mongodb://TwitterIPO?authSource=TwitterIPO'

client = pymongo.MongoClient(mongo_connect_string)
db = client.TwitterIPO
RawTweets = db.RawTweets
PriceData = db.PriceData
ProcessedTweets = db.ProcessedTweets

In [ ]:
# Get file list from 'HTML' directory
directory = 'HTML'
filelist = os.listdir(directory)
len(filelist)

In [ ]:
# Get company symbol from filename
def get_cashtag(filename):
    
    if filename.find('_') == -1:
        return filename[:filename.find('.')]
    else:
        filename = filename[:filename.find('_')]
        return filename

In [ ]:
result = []
def get_twitterdata(filename):
    
    html = open(directory + '/' + filename, encoding='utf8')
    soup = BeautifulSoup(html, 'lxml')
    
    cashtag = get_cashtag(filename)
    
    content = soup.findAll('li', class_ = 'js-stream-item stream-item stream-item ')
    
    for tweet in content:
        tweet_id = tweet.get('data-item-id')
        text = tweet.find('p', class_='TweetTextSize').text
        lang = tweet.find('p', class_='TweetTextSize').get('lang')
        datems = tweet.find('span', class_='_timestamp').get('data-time-ms')
        datepy = datetime.utcfromtimestamp(float(datems) / 1000.0)
        datepy = datepy - timedelta(hours=8)
        
        #date_str = tweet.find('a', class_='tweet-timestamp').get('title')
        #date = datetime.strptime(date_str,'%I:%M %p - %d %b %Y')        
        
        data = {}
        
        data['company'] = cashtag
        data['tweet_id'] = tweet_id
        data['text'] = str(text)
        data['datems'] = datems
        data['datepy'] = datepy
        data['lang'] = lang
        data['UID'] = str(tweet_id) + '_' + str(cashtag)
        
        #print(data)
        result.append(data)
        ProcessedTweets.insert_one(data)
        
    return result

In [ ]:
for file in filelist:
    #print('Processing File ' + file)
    get_twitterdata(file)
    print('*** Complete ' + file + ' ***')